### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-32B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/275k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.5.9 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


In [ ]:
from datasets import load_dataset
reasoning_dataset = load_dataset("LockeLamora2077/hayabusa_llm_report_forensic_reasoning", split = "train")
non_reasoning_dataset = load_dataset("LockeLamora2077/hayabusa_llm_forensic_report", split = "train")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

hayabusa_data.parquet:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/68 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

hayabusa_qa.csv:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31 [00:00<?, ? examples/s]

Let's see the structure of both datasets:

We now convert the reasoning dataset into conversational format:

In [ ]:
def generate_non_reasoning_conversation(examples):
    problems  = examples["question"]
    solutions = examples["answer"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [ ]:

non_reasoning_conversations = tokenizer.apply_chat_template(
    non_reasoning_dataset.map(generate_non_reasoning_conversation, batched = True)["conversations"],
    tokenize = False,
)


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [ ]:
non_reasoning_conversations[0]

'<|im_start|>user\nWhat is Hayabusa and who developed it?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nHayabusa is an open-source, high-performance forensic and threat hunting tool designed to analyze Windows event logs. It was developed by the Yamato Security group in Japan.<|im_end|>\n'

Let's see the first transformed row:

In [ ]:
def generate_conversation(examples):
    problems  = examples["question"]
    solutions = examples["answer_with_reasoning"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [ ]:
chat_percentage = 0.75

In [ ]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_dataset)
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations) * (1.0 - chat_percentage)),
    random_state = 2407,
)

In [ ]:
reasoning_conversations[0]

'<|im_start|>user\nWhat is Hayabusa and who developed it?<|im_end|>\n<|im_start|>assistant\n<think>\nHayabusa is an open-source, high-performance forensic and threat hunting tool designed to analyze Windows event logs. It was developed by the Yamato Security group in Japan.\n</think>\n\n Final Answer: Hayabusa is an open-source, high-performance forensic and threat hunting tool designed to analyze Windows event logs.<|im_end|>\n'

In [ ]:
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_conversations)
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
19.121 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 99 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 268,435,456/32,000,000,000 (0.84% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.053200
2,3.118700
3,3.521500
4,3.037500
5,3.074800
6,2.454200
7,2.668800
8,2.066800
9,2.125900
10,1.985100


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

118.4972 seconds used for training.
1.97 minutes used for training.
Peak reserved memory = 21.713 GB.
Peak reserved memory for training = 2.592 GB.
Peak reserved memory % of max memory = 54.89 %.
Peak reserved memory for training % of max memory = 6.553 %.


In [ ]:

    model.push_to_hub_gguf(
        "LockeLamora2077/NiNa_qw_reasoning_improved", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 19.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 59.51 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 19%|█▉        | 12/64 [00:00<00:01, 26.09it/s]
We will save to Disk and not RAM now.
100%|██████████| 64/64 [02:08<00:00,  2.01s/it]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting qwen3 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at LockeLamora2077/NiNa_qw_reasoning_improved into bf16 GGUF format.
The output location will be /content/LockeLamora2077/NiNa_qw_reasoning_improved/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: NiNa_qw_reasoning_improved
INFO:hf-to-gguf:Model architecture: Qwen3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/19.8G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/LockeLamora2077/NiNa_qw_reasoning_improved


In [ ]:
messages = [
    {"role" : "user", "content" : "What are the limitations of the IT Forensic Hayabusa Scan Tool?"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

While Hayabusa is powerful, it has limitations such as requiring proper rule configuration and system resources. It may also miss custom or obfuscated threats without tailored rules.<|im_end|>
